# Parse the XMLtv file from [XMLtv.ch](xmltv.ch)

In [3]:
from io import BytesIO
import pandas as pd
from PIL import Image
import requests
import xmltodict
import plotly.express as px

pd.options.plotting.backend = "plotly"


In [4]:
# Get the latest verion of the file
headers = {'User-Agent': 'Mozilla'}
URL = "https://xmltv.ch/xmltv/xmltv-tnt.xml"

response = requests.get(URL, headers=headers)
with open("../data/xmltv-tnt.xml", 'wb') as outfile:
    _ = outfile.write(response.content)

In [5]:
# Reading the data inside the xml
with open('../data/xmltv-tnt.xml', 'r', encoding='utf-8') as f:
    data = f.read()

# Parsing
data = xmltodict.parse(data)

In [6]:
# channels to dataframe
df_channels = pd.DataFrame(data["tv"]["channel"])
df_channels = pd.json_normalize(df_channels.to_dict(orient="records"))
df_channels.rename(columns={"@id": "channel_id", "icon.@src": "channel_icon", "display-name": "channel_name"}, inplace=True)

df_channels.head()

,channel_id,channel_name,channel_icon
0,C192.api.telerama.fr,TF1,https://television.telerama.fr/sites/tr_master...
1,C4.api.telerama.fr,France 2,https://television.telerama.fr/sites/tr_master...
2,C80.api.telerama.fr,France 3,https://television.telerama.fr/sites/tr_master...
3,C34.api.telerama.fr,Canal+,https://television.telerama.fr/sites/tr_master...
4,C111.api.telerama.fr,Arte,https://television.telerama.fr/sites/tr_master...


In [7]:
# programs to dataframe
df_programs = pd.DataFrame(data["tv"]["programme"])
df_programs = pd.json_normalize(df_programs.to_dict(orient="records"), sep="_")
# Clean column names 
df_programs.columns = [col.replace("@", "").replace("#", "").replace("-", "") for col in df_programs.columns]
# Join with df_channels 
df_programs = df_programs.join(df_channels.set_index("channel_id"), on="channel")
# Drop empty columns
df_programs.dropna(axis=1, how='all', inplace=True)
# Convert some columns to datetime
df_programs["start"] = pd.to_datetime(df_programs["start"], infer_datetime_format=True)
df_programs["stop"] = pd.to_datetime(df_programs["stop"], infer_datetime_format=True)

df_programs.head(3)

,start,stop,channel,title,subtitle,date,desc_lang,desc_text,category_lang,category_text,...,episodenum_text,audio_stereo,rating_system,rating_value,rating_icon_src,starrating_value,subtitles_type,subtitles_language,channel_name,channel_icon
0,2022-11-04 00:50:00+01:00,2022-11-04 01:35:00+01:00,C192.api.telerama.fr,Esprits criminels,Ancienne blessure,2008,fr,Saison:3 - Episode:14 - Rossi rouvre officieus...,fr,série policière,...,2.13.,bilingual,CSA,-10,http://upload.wikimedia.org/wikipedia/commons/...,NaN,NaN,NaN,TF1,https://television.telerama.fr/sites/tr_master...
1,2022-11-04 01:35:00+01:00,2022-11-04 02:30:00+01:00,C192.api.telerama.fr,Esprits criminels,Retour vers le passé,2008,fr,Saison:3 - Episode:11 - Le nouveau shérif de F...,fr,série policière,...,2.10.,bilingual,CSA,-12,http://upload.wikimedia.org/wikipedia/commons/...,NaN,NaN,NaN,TF1,https://television.telerama.fr/sites/tr_master...
2,2022-11-04 02:30:00+01:00,2022-11-04 06:25:00+01:00,C192.api.telerama.fr,Programmes de la nuit,NaN,NaN,fr,Retrouvez tous vos programmes de nuit.,fr,programme indéterminé,...,NaN,NaN,CSA,Tout public,NaN,NaN,NaN,NaN,TF1,https://television.telerama.fr/sites/tr_master...


In [8]:
df_programs.columns.tolist()

['start',
 'stop',
 'channel',
 'title',
 'subtitle',
 'date',
 'desc_lang',
 'desc_text',
 'category_lang',
 'category_text',
 'length_units',
 'length_text',
 'icon_src',
 'episodenum_system',
 'episodenum_text',
 'audio_stereo',
 'rating_system',
 'rating_value',
 'rating_icon_src',
 'starrating_value',
 'subtitles_type',
 'subtitles_language',
 'channel_name',
 'channel_icon']

In [9]:
# Compute duration in minutes
# With this we can ignore "length.units" and "length.text" which is too convoluted
df_programs["length_minutes"] = (df_programs["stop"] - df_programs["start"]).apply(lambda x: int(x.total_seconds() / 60))

In [10]:
df_programs["desc_lang"].value_counts(dropna=False)
df_programs["category_lang"].value_counts(dropna=False)
df_programs["category_text"].value_counts(dropna=False)
df_programs["date"].value_counts(dropna=False).sort_index(ascending=False)

fr     8861
NaN    1063
Name: desc_lang, dtype: int64

fr    9924
Name: category_lang, dtype: int64

série d'animation                       1337
journal                                  653
jeunesse : dessin animé dessin animé     636
météo                                    584
série humoristique                       504
                                        ... 
emission spéciale                          1
jeunesse : dessin animé jeunesse           1
sport : cyclisme                           1
film sentimental                           1
documentaire rock-pop                      1
Name: category_text, Length: 186, dtype: int64

2022     480
2021     413
2020     346
2019     290
2018     302
2017     319
2016     204
2015     143
2014     116
2013     172
2012      75
2011      57
2010     139
2009     120
2008      88
2007      13
2006      59
2005      16
2004      53
2003       5
2002       2
2001      37
2000       7
1999      27
1997       2
1996      66
1995      71
1993      24
1992      66
1991       1
1990       4
1989       4
1988       2
1985       2
1984       2
1983      18
1981       1
1980      36
1976       2
1974       1
1972       1
1969       1
1966       1
1957       1
1937       1
1933       1
NaN     6133
Name: date, dtype: int64

In [11]:
# Select only relevant columns
columns = [
    'start',
    'stop',
    'length_minutes',
    'channel_name',
    'title',
    'subtitle',
    'date',
    'desc_text',
    'category_text',
]
df = df_programs[columns]

In [37]:
import ipywidgets as widgets

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


a = widgets.Dropdown(
    options=df_channels.channel_name.unique(),
    value='TF1',
    description='Channel:',
    disabled=False,
)

def fn(a):
    print(f' Viewing data for {a}')
    df_a = df[df.channel_name == a]
    grouped = df_a.groupby("category_text").length_minutes.sum().sort_values(ascending=False)
    fig = grouped.plot.barh()
    fig.show()

out = widgets.interactive_output(fn, {'a': a})

widgets.VBox([widgets.VBox([a]), out])

In [40]:
# The widgets might not work so here's an example
selected_channel = "TF1"
df_a = df[df.channel_name == selected_channel]
grouped = df_a.groupby("category_text").length_minutes.sum().sort_values(ascending=False)
fig = grouped.plot.barh()
fig.show()

In [85]:
# Show one day of data
start_date = "2022-11-05"
end_date = "2022-11-06"

df_day = df_programs[(df_programs.start >= start_date) & (df_programs.stop < end_date)]
df_day = df_programs[(df_programs.start >= start_date) & (df_programs.start < end_date)]

df_day.groupby(["channel_name", "category_text"]).length_minutes.sum()

df_day.groupby(["category_text", "channel_name"]).length_minutes.sum().reset_index()

channel_name  category_text           
6ter          documentaire animalier      180
              documentaire téléréalité    345
              magazine culinaire          255
              magazine de télé-achat      125
              programme indéterminé       290
                                         ... 
W9            magazine de télé-achat      170
              météo                         5
              programme indéterminé       180
              série humoristique          470
              téléfilm sentimental        310
Name: length_minutes, Length: 244, dtype: int64

,category_text,channel_name,length_minutes
0,autre,Arte,0
1,autre,RMC Découverte,273
2,autre,RMC Story,170
3,clips,CSTAR,680
4,clips,M6,60
...,...,...,...
239,téléfilm sentimental,TF1 Séries Films,580
240,téléfilm sentimental,TMC,230
241,téléfilm sentimental,W9,310
242,téléréalité,TF1,275


In [92]:
df_day.groupby(["category_text", "channel_name"]).length_minutes.agg(["count", "sum"]).reset_index()

,category_text,channel_name,count,sum
0,autre,Arte,1,0
1,autre,RMC Découverte,1,273
2,autre,RMC Story,1,170
3,clips,CSTAR,7,680
4,clips,M6,1,60
...,...,...,...,...
239,téléfilm sentimental,TF1 Séries Films,6,580
240,téléfilm sentimental,TMC,2,230
241,téléfilm sentimental,W9,3,310
242,téléréalité,TF1,3,275


In [101]:
df1 = df_day.groupby(["channel_name", "category_text"]).length_minutes.agg(["count", "sum", "mean"]).reset_index()
df1
fig1 = px.bar(df1,
              x="channel_name",
              y="sum",
              color="category_text",
              title=f"Number of minutes for each channel & category for {start_date}",
              hover_data=df1.columns)
_ = fig1.add_hline(
    y=1440,
    line_dash="dot",
    annotation_text="24 hours of content", 
    annotation_position="top left",
    annotation_font_size=12,
    annotation_font_color="black"
)
_ = fig1.update_yaxes(title="")
_ = fig1.update_xaxes(title="")
fig1.show()

fig2 = px.bar(df1,
              x="category_text",
              y="sum",
              color="channel_name",
              title=f"Number of minutes for each category & channel {start_date}",
              hover_data=df1.columns)
_ = fig2.update_yaxes(title="")
_ = fig2.update_xaxes(title="")
fig2.show()

,channel_name,category_text,count,sum,mean
0,6ter,documentaire animalier,6,180,30.000000
1,6ter,documentaire téléréalité,12,345,28.750000
2,6ter,magazine culinaire,3,255,85.000000
3,6ter,magazine de télé-achat,1,125,125.000000
4,6ter,programme indéterminé,1,290,290.000000
...,...,...,...,...,...
239,W9,magazine de télé-achat,1,170,170.000000
240,W9,météo,1,5,5.000000
241,W9,programme indéterminé,1,180,180.000000
242,W9,série humoristique,3,470,156.666667


In [114]:
# Try to make sub-categories
df_programs.category_text.value_counts().index.tolist()

df_programs.category_text = df_programs.category_text.str.replace(" : ", " ")
df_programs.category_text.str.split("série ", expand=True).value_counts()
df_programs.category_text.str.split("film ", expand=True).value_counts()
df_programs.category_text.str.split("jeunesse ", expand=True).value_counts()
df_programs.category_text.str.split("divertissement ", expand=True).value_counts()
df_programs.category_text.str.split("sport ", expand=True).value_counts()
df_programs.category_text.str.split("magazine ", expand=True).value_counts()
df_programs.category_text.str.split("documentaire ", expand=True).value_counts()

["série d'animation",
 'journal',
 'jeunesse : dessin animé dessin animé',
 'météo',
 'série humoristique',
 "magazine d'actualité",
 'série policière',
 'magazine de société',
 'documentaire téléréalité',
 'divertissement',
 'divertissement : jeu',
 'documentaire société',
 'magazine politique',
 "magazine d'information",
 'clips',
 'sport : multisports',
 'téléréalité',
 'programme indéterminé',
 'série réaliste',
 "magazine de l'économie",
 'téléfilm sentimental',
 'documentaire découvertes',
 'magazine jeunesse',
 'talk-show',
 'série sentimentale',
 'magazine de télé-achat',
 'divertissement-humour',
 'magazine de reportages',
 'magazine de la gastronomie',
 'magazine musical',
 'magazine littéraire',
 'magazine de services',
 'série dramatique',
 'magazine du consommateur',
 'débat',
 'documentaire animalier',
 'jeunesse : emission jeunesse',
 'magazine culturel',
 'série fantastique',
 'documentaire sciences et technique',
 'magazine du cinéma',
 "magazine de l'art de vivre",
 '

0  1                 
   d'animation           1337
   humoristique           504
   policière              321
   réaliste               135
   sentimentale            87
   dramatique              65
   fantastique             53
   jeunesse                30
   d'aventures             18
   historique              16
   d'action                 9
   culinaire                6
   de science-fiction       6
   de téléréalité           6
   de suspense              4
   hospitalière             2
dtype: int64

0     1                        
télé  sentimental                  117
      comédie                       30
      d'action                      29
      drame                         23
télé  dramatique                    20
      d'animation                   19
      thriller                      17
      court métrage                 16
      comédie dramatique            13
      fantastique                   11
      d'aventures                    8
      de science-fiction             7
télé  romanesque                     7
      policier                       6
      de guerre                      6
      policier                       5
      court métrage d'animation      4
      d'horreur                      4
télé  humoristique                   4
      comédie romantique             3
      pour la jeunesse               3
télé  fantastique                    2
      catastrophe                    2
      western                        2
télé  érotique                  

0  1                        
   dessin animé dessin animé    636
   emission jeunesse             59
   dessin animé manga            10
   dessin animé jeunesse          1
dtype: int64

0  1  
   jeu    210
dtype: int64

0  1                  
   multisports            159
   rugby                   44
   football                29
   fitness                 18
   hippisme                13
   biathlon                10
   de force                 9
   formule 1                8
   golf                     7
   voile                    7
   mma                      5
   motocyclisme             5
   basket-ball              5
   football américain       3
   jt sport                 2
   marathon                 2
   patinage artistique      2
   tennis                   2
   handball                 1
   endurance                1
   cyclo-cross              1
   cyclisme                 1
   ski freestyle            1
   triathlon                1
dtype: int64

0  1                 
   d'actualité           348
   de société            311
   politique             195
   d'information         186
   de l'économie         129
   jeunesse               87
   de télé-achat          84
   de reportages          75
   de la gastronomie      73
   musical                71
   littéraire             67
   de services            65
   du consommateur        64
   culturel               57
   du cinéma              50
   de l'art de vivre      47
   judiciaire             46
   de découvertes         42
   éducatif               37
   culinaire              36
   sportif                34
   de l'automobile        29
   de la santé            21
   de l'environnement     18
   animalier              17
   historique             16
   religieux              15
   de la décoration       14
   régional               14
   des médias              8
   du show-biz             7
   du jardinage            6
   de la mer               6
   du tourisme       

0  1                    
   téléréalité              288
   société                  201
   découvertes               99
   animalier                 60
   sciences et technique     52
   histoire                  45
   justice                   24
   culture                   23
   nature                    20
   aventures                 19
   civilisations             17
   cinéma                    14
   gastronomie               12
   sport                     11
   environnement             11
   pêche                     11
   musique                   10
   voyage                    10
   politique                  7
   santé                      6
   beaux-arts                 5
   art de vivre               4
   education                  3
   fiction                    2
   autre                      2
   lettres                    1
   musique classique          1
   géopolitique               1
   rock-pop                   1
dtype: int64

In [121]:
def removeprefix(text):
    for prefix in ["de ", "de la ", "du ", "de l'", "d'"]:   
        if text.startswith(prefix):
            return text[len(prefix):]
        return text

cat_prefixes = [
    "série",
    "film",
    "jeunesse",
    "divertissement",
    "sport",
    "magazine",
    "documentaire",
]

def split_category(cat_text):
    main_cat = cat_text
    sub_cat = None
    
    for cat_prefix in cat_prefixes:
        if cat_text.startswith(cat_prefix + " "):
            main_cat = cat_prefix
            sub_cat = cat_text.split(cat_prefix + " ")[1].removeprefix()
        
            return main_cat, sub_cat
    
    return main_cat, sub_cat

In [120]:
df_programs.category_text.str.split("film ", expand=True)[1].value_counts()

cat_text = df_programs.category_text.tolist()

for split_category in ["film"]:
    if split_category + " " in cat_text:
        main_cat = split_category
        sub_cat = cat_text.split(split_category + " ")[1].removeprefix()

sentimental                  118
comédie                       30
d'action                      29
drame                         23
d'animation                   20
dramatique                    20
thriller                      17
court métrage                 16
comédie dramatique            13
fantastique                   13
policier                      11
d'aventures                    8
romanesque                     7
de science-fiction             7
de guerre                      6
catastrophe                    4
court métrage d'animation      4
humoristique                   4
d'horreur                      4
pour la jeunesse               3
comédie romantique             3
érotique                       2
western                        2
comédie sentimentale           2
pornographique                 1
biographie                     1
drame historique               1
documentaire                   1
court métrage burlesque        1
court métrage dramatique       1
Name: 1, d